Import Modules

In [1]:
import numpy as np
import cv2
import glob
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms as T
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


데이터 전처리

In [2]:
transform = T.Compose([T.Resize(256), T.RandomCrop(224), T.ToTensor()])

In [3]:
class Dataset(Dataset):
    def __init__(self, path='train'):
        super(Dataset, self).__init__()

        self.img_path = []
        bo_path = glob.glob(path+'/bo/*.jpg')
        notbo_path = glob.glob(path+'/not_bo/*.jpg')
        self.img_path= bo_path + notbo_path

        self.label_list = [1]*len(bo_path) + [0]*len(notbo_path)

    def __getitem__(self, index):
        img = cv2.imread(self.img_path[index])

        img_pil = Image.fromarray(img)
        self.img_tensor = transform(img_pil)

        self.label_tensor = torch.tensor(self.label_list[index])

        return self.img_tensor.to(device), self.label_tensor.to(device)

    def __len__(self):
        return len(self.img_path)


In [4]:
training_dataset = Dataset('train')
training_loader = DataLoader(dataset=training_dataset, batch_size=8, shuffle=True)

In [5]:
validation_dataset = Dataset('train')
validation_loader = DataLoader(dataset=validation_dataset, batch_size=8, shuffle=True)

In [6]:
vgg16 = models.vgg16(pretrained=True)

vgg16 = vgg16.to(device)

In [7]:
for param in vgg16.parameters():
    param.requires_grad = False

In [8]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg16.parameters(), lr=0.001)

Training

In [9]:
for epoch in range(20):
    loss_val = 0
    for itr, data in enumerate(training_loader):
        optimizer.zero_grad()

        inputs, labels = data

        pred = vgg16(inputs)
        loss = loss_function(pred, labels)

        loss.backward()
        optimizer.step()

        loss_val += loss.item()

    print("Epoch:", epoch+1, ",  Loss:", loss_val)

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

prediction and evaluation

In [ ]:
pred_list = []
label_list = []

for itr, data in enumerate(validation_loader):
    inputs, labels = data

    pred = vgg16(inputs)
    pred_category = torch.argmax(pred, dim=1)

    pred_list = pred_list + list(pred_category)
    label_list = label_list + list(labels)

accu = np.mean(np.array(pred_list) == np.array(label_list))
print("validation accuracy:", accu)